In [24]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.patches import Rectangle, Polygon
from IPython.display import clear_output
import torch
import warnings
warnings.filterwarnings('ignore')
import datetime
import csv
import os 
import sys
import pandas as pd
import gym
import tarfile

File to write down actor and critic losses.

In [25]:
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")  # Format the current time
output_dir = f'./output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
loss_file = open(f'{output_dir}/losses_seed_{current_time}.csv', mode='w', newline='')
loss_writer = csv.writer(loss_file)
loss_writer.writerow(['iteration', 'actor_loss', 'critic_loss'])

34

In [26]:
def calculate_power_with_wraparound(current, previous, time_diff, wraparound_value=262143.328850):
    diff = current - previous
    if diff < 0:  # Wraparound detected
        diff = (wraparound_value - previous) + current
    return diff / time_diff


In [27]:
def compute_power(pubEnergy):
    power = {}
    geopm_sensor0 = geopm_sensor1 = pd.DataFrame({'timestamp':[],'value':[]})
    for i,row in pubEnergy.iterrows():
        if i%2 == 0:
            geopm_sensor0 = pd.concat([geopm_sensor0, pd.DataFrame([{'timestamp': row['time'], 'value': row['value']}])], ignore_index=True)
        else:
            geopm_sensor1 = pd.concat([geopm_sensor1, pd.DataFrame([{'timestamp': row['time'], 'value': row['value']}])], ignore_index=True)


    power['geopm_power_0'] = pd.DataFrame({
        'timestamp': geopm_sensor0['timestamp'][1:],  # Add timestamps
        'power': [
            calculate_power_with_wraparound(
                geopm_sensor0['value'][i],
                geopm_sensor0['value'][i-1],
                geopm_sensor0['timestamp'][i] - geopm_sensor0['timestamp'][i-1]
            ) for i in range(1, len(geopm_sensor0))
        ]
    })

    # Apply the same logic to geopm_power_1
    power['geopm_power_1'] = pd.DataFrame({
        'timestamp': geopm_sensor1['timestamp'][1:],  # Add timestamps
        'power': [
            calculate_power_with_wraparound(
                geopm_sensor1['value'][i],
                geopm_sensor1['value'][i-1],
                geopm_sensor1['timestamp'][i] - geopm_sensor1['timestamp'][i-1]
            ) for i in range(1, len(geopm_sensor1))
        ]
    })

    min_length = min(len(power['geopm_power_0']), len(power['geopm_power_1']))
    geopm_power_0 = power['geopm_power_0'][:min_length]
    geopm_power_1 = power['geopm_power_1'][:min_length]

    average_power = pd.DataFrame({
        'timestamp': geopm_power_0['timestamp'],  # Use the timestamp from geopm_power_0
        'average_power': [(p0 + p1) / 2 for p0, p1 in zip(geopm_power_0['power'], geopm_power_1['power'])]
    })
    average_power['elapsed_time'] = average_power['timestamp'] - average_power['timestamp'].iloc[0]
    power['average_power'] = average_power
    return power

In [28]:
def measure_progress(progress_data, energy_data):
    Progress_DATA = {} 
    progress_sensor = pd.DataFrame(progress_data)
    first_sensor_point = min(energy_data['average_power']['timestamp'].iloc[0], progress_sensor['time'][0])
    progress_sensor['elapsed_time'] = progress_sensor['time'] - first_sensor_point  # New column for elapsed time
    # progress_sensor = progress_sensor.set_index('elapsed_time')
    performance_elapsed_time = progress_sensor.elapsed_time
    # Add performance_frequency as a new column in progress_sensor
    frequency_values = [
        progress_data['value'].iloc[t] / (performance_elapsed_time[t] - performance_elapsed_time[t-1]) for t in range(1, len(performance_elapsed_time))
    ]
    
    # Ensure the frequency_values length matches the index length
    frequency_values = [0] + frequency_values  # Prepend a 0 for the first index
    progress_sensor['frequency'] = frequency_values
    upsampled_timestamps= energy_data['average_power']['timestamp']
    
    # true_count = (progress_sensor['time'] <= upsampled_timestamps.iloc[0]).sum()

    progress_frequency_median = pd.DataFrame({'median': np.nanmedian(progress_sensor['frequency'].where(progress_sensor['time'] <= upsampled_timestamps.iloc[0])), 'timestamp': upsampled_timestamps.iloc[0]}, index=[0])
    for t in range(1, len(upsampled_timestamps)):
        progress_frequency_median = pd.concat([progress_frequency_median, pd.DataFrame({'median': [np.nanmedian(progress_sensor['frequency'].where((progress_sensor['time'] >= upsampled_timestamps.iloc[t-1]) & (progress_sensor['time'] <= upsampled_timestamps.iloc[t])))],
        'timestamp': [upsampled_timestamps.iloc[t]]})], ignore_index=True)
    progress_frequency_median['elapsed_time'] = progress_frequency_median['timestamp'] - progress_frequency_median['timestamp'].iloc[0]
    # Assign progress_frequency_median as a new column
    Progress_DATA['progress_sensor'] = progress_sensor
    Progress_DATA['progress_frequency_median'] = progress_frequency_median
    return Progress_DATA

In [29]:
def collect_papi(PAPI_data):
    PAPI = {}
    for scope in PAPI_data['scope'].unique():
        # Extract the string between the 3rd and 4th dots
        scope_parts = scope.split('.')
        if len(scope_parts) > 4:  # Ensure there are enough parts
            extracted_scope = scope_parts[3]
            # Aggregate the data for the extracted scope using pd.concat
            PAPI[extracted_scope] = PAPI_data[PAPI_data['scope'] == scope]
            instantaneous_values = [0] + [PAPI[extracted_scope]['value'].iloc[k] - PAPI[extracted_scope]['value'].iloc[k-1] for k in range(1,len(PAPI[extracted_scope]))]
            # Normalize the instantaneous values between 0 and 10
            # min_val = min(instantaneous_values)
            # max_val = max(instantaneous_values)
            PAPI[extracted_scope]['instantaneous_value'] = instantaneous_values
            PAPI[extracted_scope]['elapsed_time'] = PAPI[extracted_scope]['time'] - PAPI[extracted_scope]['time'].iloc[0]
    return PAPI

In [30]:
def derived_papi(PAPI_data):
    DERIVED = {}
    DERIVED['TOT_INS_PER_CYC'] = np.array(PAPI_data['PAPI_TOT_INS']['instantaneous_value']) / np.array(PAPI_data['PAPI_TOT_CYC']['instantaneous_value'])
    DERIVED['TOT_CYC_PER_INS'] = np.array(PAPI_data['PAPI_TOT_CYC']['instantaneous_value']) / np.array(PAPI_data['PAPI_TOT_INS']['instantaneous_value'])
    DERIVED['L3_TCM_PER_TCA'] = np.array(PAPI_data['PAPI_L3_TCM']['instantaneous_value']) / np.array(PAPI_data['PAPI_L3_TCA']['instantaneous_value'])  
    DERIVED['TOT_STL_PER_CYC'] = np.array(PAPI_data['PAPI_RES_STL']['instantaneous_value']) / np.array(PAPI_data['PAPI_TOT_CYC']['instantaneous_value']) 
    DERIVED['TIME_STAMP'] = np.array(PAPI_data['PAPI_TOT_INS']['elapsed_time'])
    return DERIVED

In [43]:
def normalize(traces):
    normalized_PAPI = {}
    max_value = {'PAPI_L3_TCA': float('-inf'), 'PAPI_TOT_INS': float('-inf'), 'PAPI_TOT_CYC': float('-inf'), 'PAPI_RES_STL': float('-inf'), 'PAPI_L3_TCM': float('-inf')}
    min_value = {'PAPI_L3_TCA': float('inf'), 'PAPI_TOT_INS': float('inf'), 'PAPI_TOT_CYC': float('inf'), 'PAPI_RES_STL': float('inf'), 'PAPI_L3_TCM': float('inf')}
    for app in traces.keys():
        for trace in traces[app].keys():
            for scope in traces[app][trace]['papi'].keys():
                max_value[scope] = max(max_value[scope],max(traces[app][trace]['papi'][scope]['instantaneous_value']))
                min_value[scope] = min(min_value[scope],min(traces[app][trace]['papi'][scope]['instantaneous_value']))
    for app in traces.keys():
        for trace in traces[app].keys():
            for scope in traces[app][trace]['papi'].keys():
                traces[app][trace]['papi'][scope]['normalized_value'] = [(value - min_value[scope]) / (max_value[scope] - min_value[scope]) * 10 for value in traces[app][trace]['papi'][scope]['instantaneous_value']]
    return traces

def generate_PCAP(PCAP_data):
    for row in PCAP_data.iterrows():
        if row[1]['time'] == 0:
            PCAP_data = PCAP_data.drop(row[0])


    PCAP_data['elapsed_time'] = PCAP_data['time'] - PCAP_data['time'].iloc[0]
    return PCAP_data

def get_data_dir():
    # current_dir = os.path.dirname(os.path.abspath(__file__))
    current_dir = os.getcwd()
    print(current_dir)
    return os.path.join(current_dir, "experiment_data", "data_generation")

In [44]:
DATA_DIR = get_data_dir()
root,folders,files = next(os.walk(DATA_DIR))
training_data = {}
for APP in folders:
    APP_DIR = os.path.join(DATA_DIR, APP)
    training_data[APP] = {}
    for file in next(os.walk(APP_DIR))[2]:
        training_data[APP][file] = {}
        if file.endswith('.tar'):
            tar_path = os.path.join(APP_DIR, file)
            extract_dir = os.path.join(APP_DIR, file[:-4])  
            
            if not os.path.exists(extract_dir):
                os.makedirs(extract_dir)
            
            with tarfile.open(tar_path, 'r') as tar:
                tar.extractall(path=extract_dir)
            
        pubProgress = pd.read_csv(f'{extract_dir}/progress.csv')
        pubEnergy = pd.read_csv(f'{extract_dir}/energy.csv')
        pubPAPI = pd.read_csv(f'{extract_dir}/papi.csv')
        pubPCAP = pd.read_csv(f'{extract_dir}/PCAP_file.csv')
        # with open(f'{extract_dir}/parameters.yaml', 'r') as f:
        #     yaml = YAML(typ='safe', pure=True)
        #     parameters = yaml.load(f)
        #     PCAP = parameters['PCAP']
        # training_data['data']['PCAP'] = pd.read_csv(f'{extract_dir}/PCAP_file.csv')
        training_data[APP][file]['power'] = compute_power(pubEnergy)
        training_data[APP][file]['progress'] = measure_progress(pubProgress,training_data[APP][file]['power'])
        training_data[APP][file]['papi'] = collect_papi(pubPAPI)
        training_data[APP][file]['PCAP'] = generate_PCAP(pubPCAP)
        training_data[APP][file]['derived_papi'] = derived_papi(training_data[APP][file]['papi'])   
        # print(training_data[APP][file]['PCAP'] )    
# training_data = normalize(training_data)

/Users/akhileshraj/Desktop/summer2024/main_codes


In [42]:
# training_data[APP][file]['papi']
# np.array(training_data[APP][file]['papi']['PAPI_TOT_CYC']['instantaneous_value']) / np.array(training_data[APP][file]['papi']['PAPI_TOT_INS']['instantaneous_value'])
# np.array(training_data[APP][file]['papi']['PAPI_TOT_INS']['instantaneous_value']) / np.array(training_data[APP][file]['papi']['PAPI_TOT_CYC']['instantaneous_value'])
# np.array(training_data[APP][file]['papi']['PAPI_L3_TCM']['instantaneous_value']) / np.array(training_data[APP][file]['papi']['PAPI_L3_TCA']['instantaneous_value'])
# np.array(training_data[APP][file]['papi']['PAPI_RES_STL']['instantaneous_value']) / np.array(training_data[APP][file]['papi']['PAPI_TOT_CYC']['instantaneous_value'])
APP = 'ones-stream-add'
file = list(training_data[APP].keys())[0]

training_data[APP][file]['derived_papi']


{'TOT_INS_PER_CYC': array([       nan, 0.14205005, 0.1420557 , 0.14185485, 0.14107412,
        0.14100823, 0.14106961, 0.14073528, 0.141457  , 0.14166151,
        0.14204819, 0.14194053, 0.14161753, 0.14176039, 0.14189252,
        0.14175197, 0.14186593, 0.14166507, 0.14160525, 0.14162423,
        0.14154042, 0.14170791, 0.14145823, 0.14176053, 0.14184874,
        0.14174819, 0.14152713, 0.14174298, 0.14161214, 0.1418489 ,
        0.14176668, 0.14175412, 0.14158147, 0.1420234 , 0.14177071,
        0.1416144 , 0.14154172, 0.14179025, 0.1415452 , 0.14169732,
        0.14171472, 0.14160207, 0.14161833, 0.14179675, 0.14168404,
        0.14164377, 0.14173407, 0.14165993, 0.14161189, 0.14171468,
        0.14164428, 0.14157307, 0.14167099, 0.14166712, 0.14161321,
        0.14156901, 0.14170521, 0.14155466, 0.14170473, 0.14176603,
        0.14203591, 0.14155918, 0.14165992, 0.1416785 , 0.14151026,
        0.14159084, 0.14162665, 0.14165089, 0.14177625, 0.14155375,
        0.14163959, 0.1416955

In [ ]:

training_dataset = []
old_PCAP = 0
action_set = []
def get_roi_data(df, time_column, start_time, end_time):
    return df[(df[time_column] > start_time) & (df[time_column] <= end_time)]

def get_state(training_data, app, trace, start_time, end_time):
    ROI_progress = get_roi_data(training_data[app][trace]['progress']['progress_frequency_median'], 'timestamp', start_time, end_time)
    ROI_measured_power = get_roi_data(training_data[app][trace]['power']['average_power'], 'timestamp', start_time, end_time)
    ROI_derived_1= get_roi_data(training_data[app][trace]['derived_papi']['TOT_INS_PER_CYC'], 'TIME_STAMP', start_time, end_time)
    ROI_derived_2 = get_roi_data(training_data[app][trace]['derived_papi']['L3_TCM_PER_TCA'], 'TIME_STAMP', start_time, end_time)
    ROI_derived_3 = get_roi_data(training_data[app][trace]['derived_papi']['TOT_STL_PER_CYC'], 'TIME_STAMP', start_time, end_time)

    return (
        ROI_progress['median'].mean() if not ROI_progress.empty else 0,
        ROI_measured_power['average_power'].mean() if not ROI_measured_power.empty else 0,
        ROI_derived_1.mean() if not ROI_derived_1.empty else 0,
        ROI_derived_2.mean() if not ROI_derived_1.empty else 0,
        ROI_derived_3.mean() if not ROI_derived_1.empty else 0,
    )

# state_definition = [progress, measured_power, previous_PCAP, 'PAPI_L3_TCA', 'PAPI_TOT_INS', 'PAPI_TOT_CYC', 'PAPI_RES_STL', 'PAPI_L3_TCM']
initial_progress = 0
initial_power = 40
initial_PAPI = np.zeros(5)
state = (initial_progress,initial_power,initial_PAPI)
t1 = float('-inf')
for app in training_data.keys():
    for trace in training_data[app].keys():
        pcap_data = training_data[app][trace]['PCAP']
        for i, row in pcap_data.iterrows():
            t2 = row['time']
            
            # Get current state
            state = get_state(training_data, app, trace, t1, t2)
            
            # Get next state (look ahead to next row)
            if i + 1 < len(pcap_data):
                t3 = pcap_data.iloc[i + 1]['time']
                next_state = get_state(training_data, app, trace, t2, t3)
            else:
                next_state = state  # Use current state if it's the last row
            
            action = row['value']  # Assuming PCAP is in the 'value' column
            
            # Calculate the reward
            reward = env.reward(state[0], action, next_state[0], state[1])
            
            # Add to training dataset
            training_dataset.append((state, action, reward, next_state))
            
            t1 = t2


# Define the CSV file name and path
csv_file_name = 'training_dataset.csv'
csv_file_path = os.path.join(DATA_DIR, csv_file_name)

In [11]:

T_S = 1
ACTIONS = [78.0, 83.0, 89.0, 95.0, 101.0, 107.0, 112.0, 118.0, 124.0, 130.0, 136.0, 141.0, 147.0, 153.0, 159.0, 165.0]
exec_steps = 10000    
TOTAL_ACTIONS = len(ACTIONS)                                                                                                  # Total clock cycles needed for the execution of program.
ACTION_MIN = min(ACTIONS)                                                                                                    # Minima of control space
ACTION_MAX = max(ACTIONS)                                                                                                     # Maxima of control space
ACT_MID = ACTION_MIN + (ACTION_MAX - ACTION_MIN) / 2                                                                    # Midpoint of the control space to compute the normalized action space                           
OBS_MIN = np.zeros((5,))  # Shape should be (7,)
OBS_MAX = np.array([300,165,1,1,1])                                                                                 # Minima of observation space
OBS_MID = OBS_MIN + (OBS_MAX - OBS_MIN) / 2
EXEC_ITERATIONS = 10000
TOTAL_OBS = OBS_MAX - OBS_MIN
OBS_ONEHOT = 'onehot'
OBS_RANDOM = 'random'
OBS_SMOOTH = 'smooth'

In [12]:
class SYS(object):
    def __init__(self,observation_type=OBS_ONEHOT,dim_obs=5,teps=0.0):
        super(SYS,self).__init__()
        self.num_actions = TOTAL_ACTIONS
        self.action_space = gym.spaces.Discrete(len(ACTIONS)) 
        self.actions = ACTIONS  
        self.observation_space = gym.spaces.Box(low=OBS_MIN, high=OBS_MAX, shape=(5,), dtype=np.float32)  # Infinite observation space with 8 dimensions
        self.dim_obs = dim_obs
    
    def reward(self, s, a, ns, measured_power):
        """ 
        Returns the reward (float)
        """
        if ns > 0:
            reward = ns/(a**2+1)
        else:
            reward = -100
        return reward

In [13]:
weighting_only = False
dataset_composition = 'random'
dataset_size = 1000
env_type = 'random'
env = SYS(observation_type=env_type, dim_obs=5, teps=0)
training_dataset = []
old_PCAP = 0
action_set = []

In [14]:
def stack_observations(env):
    obs = []
    for s in range(env.num_states):
        obs.append(env.observation(s))
    return np.stack(obs)

In [15]:
class FCNetwork(torch.nn.Module):
  def __init__(self, env, layers=[20,20]):
    super(FCNetwork, self).__init__()
    # self.all_observations = torch.tensor(stack_observations(env), dtype=torch.float32)
    dim_input = env.dim_obs
    dim_output = env.num_actions
    net_layers = []

    dim = dim_input
    for i, layer_size in enumerate(layers):
      net_layers.append(torch.nn.Linear(dim, layer_size))
      net_layers.append(torch.nn.ReLU())
      dim = layer_size
    net_layers.append(torch.nn.Linear(dim, dim_output))
    self.layers = net_layers
    self.network = torch.nn.Sequential(*net_layers)

  # def forward(self, states):
  #   observations = torch.index_select(self.all_observations, 0, states)
  #   return self.network(observations)

In [16]:
def get_tensors(list_of_tensors, list_of_indices):
  s, a, ns, r = [], [], [], []
  for idx in list_of_indices:
    s.append(list_of_tensors[idx][0])
    a.append(list_of_tensors[idx][1])
    r.append(list_of_tensors[idx][2])
    ns.append(list_of_tensors[idx][3])
  s = np.array(s)
  a = np.array(a)
  ns = np.array(ns)
  r = np.array(r)
  return s, a, ns, r

In [17]:
def project_qvalues_cql_sampled(env, s, a, target_values, network, optimizer, cql_alpha=0.1, num_steps=50, weights=None):
    # train with a sampled dataset
    target_qvalues = torch.tensor(target_values, dtype=torch.float32)
    s = torch.tensor(s, dtype=torch.int64)
    a = torch.tensor(a, dtype=torch.int64)
    pred_qvalues = network(s)
    logsumexp_qvalues = torch.logsumexp(pred_qvalues, dim=-1)

    pred_qvalues = pred_qvalues.gather(1, a.reshape(-1,1)).squeeze()
    cql_loss = logsumexp_qvalues - pred_qvalues

    loss = torch.mean((pred_qvalues - target_qvalues)**2)
    loss = loss + cql_alpha * torch.mean(cql_loss)

    network.zero_grad()
    loss.backward()
    optimizer.step()

    pred_qvalues = network(torch.arange(env.num_states))
    return pred_qvalues.detach().numpy()

In [18]:
def project_qvalues_cql(q_values, network, optimizer, num_steps=50, cql_alpha=0.1, weights=None):
    # regress onto q_values (aka projection)
    q_values_tensor = torch.tensor(q_values, dtype=torch.float32)
    for _ in range(num_steps):
       # Eval the network at each state
      pred_qvalues = network(torch.arange(q_values.shape[0]))
      if weights is None:
        loss = torch.mean((pred_qvalues - q_values_tensor)**2)
      else:
        loss = torch.mean(weights*(pred_qvalues - q_values_tensor)**2)

      # Add cql_loss
      # You can have two variants of this loss, one where data q-values
      # also maximized (CQL-v2), and one where only the large Q-values
      # are pushed down (CQL-v1) as covered in the tutorial
      cql_loss = torch.logsumexp(pred_qvalues, dim=-1, keepdim=True) # - pred_qvalues
      loss = loss + cql_alpha * torch.mean(weights * cql_loss)
      network.zero_grad()
      loss.backward()
      optimizer.step()
    return pred_qvalues.detach().numpy()

In [19]:
def q_backup_sparse_sampled(env, q_values, s, a, ns, r, discount=0.99):
  q_values_ns = q_values[ns, :]
  values = np.max(q_values_ns, axis=-1)
  target_value = r + discount * values
  return target_value


In [20]:
def conservative_q_iteration(env,
                             network,
                             num_itrs=100,
                             project_steps=50,
                             cql_alpha=0.1,
                             render=False,
                             weights=None,
                             sampled=False,
                             training_dataset=None,
                             **kwargs):
  """
  Runs Conservative Q-iteration.

  Args:
    env: A GridEnv object.
    num_itrs (int): Number of FQI iterations to run.
    project_steps (int): Number of gradient steps used for projection.
    cql_alpha (float): Value of weight on the CQL coefficient.
    render (bool): If True, will plot q-values after each iteration.
    sampled (bool): Whether to use sampled datasets for training or not.
    training_dataset (list): list of (s, a, r, ns) pairs
  """

  optimizer = torch.optim.Adam(network.parameters(), lr=1e-3)
  weights_tensor = None
  if weights is not None:
    weights_tensor = torch.tensor(weights, dtype=torch.float32)

  q_values = np.zeros((dS, dA)) #Initializing the Q-values for getting the target values
  for i in range(num_itrs):
    for j in range(project_steps):
      training_idx = np.random.choice(np.arange(len(training_dataset)), size=128)
      s, a, ns, r = get_tensors(training_dataset, training_idx)
      target_values = q_backup_sparse_sampled(env, q_values, s, a, ns, r, **kwargs)
      intermed_values = project_qvalues_cql_sampled(
          env, s, a, target_values, network, optimizer,
          cql_alpha=cql_alpha, weights=None,
      )
      if j == project_steps - 1:
        q_values = intermed_values
  return q_values

In [22]:
#@title Run conservative Q-iteration (or CQL) with finite data

# Use a tabular or feedforward NN approximator
network = FCNetwork(env, layers=[20, 20])
#network = TabularNetwork(env)

cql_alpha_val = 0.1 # @param {type:"slider", min:0.0, max:10.0, step:0.01}
weights = None
print (weighting_only)
# Run Q-iteration
q_values = conservative_q_iteration(env, network,
                                    num_itrs=100, discount=0.95, cql_alpha=cql_alpha_val,
                                    weights=weights, render=False,
                                    sampled=not(weighting_only),
                                    training_dataset=training_data)

# Compute and plot the value function
v_values = np.max(q_values, axis=1)
# plot_s_values(env, v_values, title='Values')

False


KeyboardInterrupt: 